<a href="https://colab.research.google.com/github/AlexandreOrmandes/Regress-o_II/blob/main/Mod13_Tarefa01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import patsy
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm


# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [15]:
df['log_renda'] = np.log(df['renda'])

# Verificar quais são as categorias mais frequentes para as variáveis categóricas
categorical_columns = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

most_frequent_categories = {}
for col in categorical_columns:
    most_frequent_categories[col] = df[col].value_counts().idxmax()

most_frequent_categories

{'sexo': 'F',
 'posse_de_veiculo': 'False',
 'posse_de_imovel': 'True',
 'tipo_renda': 'Assalariado',
 'educacao': 'Secundário',
 'estado_civil': 'Casado',
 'tipo_residencia': 'Casa'}

In [17]:
# Criar a fórmula para o modelo de regressão
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa")) + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'

# Ajustar o modelo
model = smf.ols(formula, data=df).fit()

# Obter o resumo estatístico do modelo
model_summary = model.summary()
model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 17 Oct 2023   Prob (F-statistic):               0.00
Time:                        21:34:05   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment("F"))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0829      0.014     -5.926      0.000      -0.110      -0.055
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment("Secundário"))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment("Casa"))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

# Avaliação dos Parâmetros e Sentido Prático
Sexo: O coeficiente para sexo masculino é significativo e positivo. Isso indica que, mantendo todas as outras variáveis constantes, ser do sexo masculino está associado a um logaritmo de renda mais elevado em comparação ao sexo feminino.

Posse de Veículo: Ter um veículo parece estar associado a um logaritmo de renda mais elevado, o que faz sentido prático.

Posse de Imóvel: Não possuir um imóvel está associado a um logaritmo de renda mais baixo, o que também é intuitivo.

Tipo de Renda: Ser empresário está associado a um logaritmo de renda mais elevado em comparação a ser assalariado, o que é esperado.

Idade e Tempo de Emprego: Ambos têm coeficientes positivos, indicando que quanto mais velho e quanto mais tempo em um emprego, maior é o logaritmo da renda, o que faz sentido prático.

Quantidade de Pessoas na Residência: O coeficiente é positivo, o que é um pouco contraintuitivo, pois esperaríamos que mais pessoas na mesma residência pudessem diluir a renda.

In [18]:
# Removendo a variável menos significativa: 'tipo_residencia'
formula = '''log_renda ~ C(sexo, Treatment("F")) + C(posse_de_veiculo, Treatment("False")) +
             C(posse_de_imovel, Treatment("True")) + C(tipo_renda, Treatment("Assalariado")) +
             C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) +
             qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'''

# Ajustando o modelo
model_simplified = smf.ols(formula=formula, data=df).fit()

# Resumo do modelo
model_simplified.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Tue, 17 Oct 2023   Prob (F-statistic):               0.00
Time:                        21:40:51   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6134      0.219     30.247      0.000       6.185       7.042
C(sexo, Treatment("F"))[T.M]                                    0.7893      0.015     53.999      0.000       0.761       0.818
C(posse_de_veiculo, Treatment("False"))[T.True]                 0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel, Treatment("True"))[T.False]                 -0.0819      0.014     -5.966      0.000      -0.109      -0.055
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment("Secundário"))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                     -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                           0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:      

R-quadrado e R-quadrado ajustado: Ambos os valores são aproximadamente 0.357 e 0.356, respectivamente. Estes são quase idênticos aos do modelo anterior, o que sugere que a remoção da variável menos significativa ('tipo_residencia') não teve um grande impacto no poder explicativo do modelo.

P-value: A maioria das variáveis ainda parece ser significativa com p-values abaixo de 0.05, exceto algumas categorias nas variáveis 'tipo_renda', 'educacao' e 'estado_civil'.

Coeficientes: Os coeficientes não mudaram significativamente em comparação com o modelo anterior.

Significância: A variável 'tipo_renda' com a categoria 'Bolsista' ainda tem um p-value alto, sugerindo que ela pode ser a próxima candidata a ser removida para simplificar o modelo.

In [20]:
# Ajustando o modelo inicial
model_initial = smf.ols(formula=formula, data=df).fit()

# Extraindo o resumo estatístico do modelo inicial
summary_initial = model_initial.summary()

In [27]:
# Função para remover a variável menos significativa do modelo
def remove_least_significant_var_optimized(model):
    # Obtém o DataFrame de p-values, exceto o intercepto
    p_values = model.pvalues[1:]

    # Encontra a variável menos significativa (maior p-value)
    remove_var = p_values.idxmax()

    # Exclui a variável menos significativa do DataFrame de design da matriz X
    X = model.model.exog
    X_df = pd.DataFrame(X, columns=model.model.exog_names)
    X_df.drop(columns=[remove_var], inplace=True)

    # Ajusta o novo modelo sem a variável menos significativa
    model = sm.OLS(model.model.endog, X_df).fit()

    return model

# Loop para remover iterativamente a variável menos significativa até que todas as variáveis restantes sejam significativas
model = model_initial
while True:
    p_values = model.pvalues[1:]  # Ignora o intercepto
    max_p_value = p_values.max()

    # Verifica se o p-value máximo é maior que 0.05
    if max_p_value > 0.05:
        model = remove_least_significant_var_optimized(model)
    else:
        break

# Extraindo o resumo estatístico do modelo final
summary_final_optimized = model.summary()
summary_final_optimized


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     530.1
Date:                Tue, 17 Oct 2023   Prob (F-statistic):               0.00
Time:                        22:08:35   Log-Likelihood:                -13572.
No. Observations:               12427   AIC:                         2.717e+04
Df Residuals:                   12413   BIC:                         2.728e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5987      0.218     30.210      0.000       6.171       7.027
C(sexo, Treatment("F"))[T.M]                                    0.7897      0.015     54.092      0.000       0.761       0.818
C(posse_de_veiculo, Treatment("False"))[T.True]                 0.0455      0.014      3.230      0.001       0.018       0.073
C(posse_de_imovel, Treatment("True"))[T.False]                 -0.0833      0.014     -6.077      0.000      -0.110      -0.056
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1550      0.015     10.413      0.000       0.126       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0591      0.022      2.661      0.008       0.016       0.103
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1096      0.014      8.078      0.000       0.083       0.136
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3313      0.111      2.979      0.003       0.113       0.549
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2709      0.109      2.489      0.013       0.058       0.484
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3713      0.116      3.213      0.001       0.145       0.598
qtd_filhos                                                     -0.2657      0.108     -2.462      0.014      -0.477      -0.054
idade                                                           0.0054      0.001      6.939      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.468      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2931      0.108      2.721      0.007       0.082       0.504
==============================================================================
Omnibus:                        0.824   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.808
Skew:                           0.019   Prob(JB):                        0.668
Kurtosis:                       3.010   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
summary_initial

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Tue, 17 Oct 2023   Prob (F-statistic):               0.00
Time:                        21:52:51   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6134      0.219     30.247      0.000       6.185       7.042
C(sexo, Treatment("F"))[T.M]                                    0.7893      0.015     53.999      0.000       0.761       0.818
C(posse_de_veiculo, Treatment("False"))[T.True]                 0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel, Treatment("True"))[T.False]                 -0.0819      0.014     -5.966      0.000      -0.109      -0.055
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment("Secundário"))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment("Casado"))[T.Separado]                0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment("Casado"))[T.Solteiro]                0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                     -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                           0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:      

# Análise do Modelo Final
O modelo final inclui apenas as variáveis significativas, com p-valores menores que 0.05.

Vamos comparar alguns dos principais indicadores dos dois modelos:

Modelo Inicial
R2 ajustado: 0.356
AIC: 27190
BIC: 27370

Modelo Final
R2 ajustado: 0.356
AIC: 27170
BIC: 27280

# Parâmetros Estimados
ntercepto (6.5987): Similar ao modelo anterior, este é o log da renda média para o grupo de referência.

Sexo (0.7897): Novamente, ser do sexo masculino está associado a um aumento no log da renda em relação às mulheres.

Posse de Veículo (0.0455): O coeficiente aumentou ligeiramente em relação ao modelo anterior, indicando um efeito positivo, ainda que pequeno, sobre o log da renda para aqueles que possuem um veículo.

Posse de Imóvel (-0.0833): A posse de imóvel continua a mostrar um efeito negativo sobre a renda, e esse efeito é ligeiramente mais forte neste modelo.

Tipo de Renda - Empresário (0.1550) e Servidor público (0.0591): Ambos estão associados a um aumento no log da renda em relação ao grupo de referência (Assalariado).

Educação - Superior completo (0.1096): Aumento no log da renda para aqueles com ensino superior completo em relação ao ensino médio.

Estado Civil - Separado (0.3313), Solteiro (0.2709), Viúvo (0.3713): Todos mostram um aumento no log da renda em relação ao estado civil de "Casado".

Quantidade de Filhos (-0.2657): Similar ao modelo anterior, mais filhos estão associados a uma diminuição no log da renda.

Idade (0.0054): Aumento ligeiramente mais forte neste modelo, mostrando que a idade continua a ter um efeito positivo sobre a renda.

Tempo de Emprego (0.0617): Sem alteração significativa, ainda mostra um forte efeito positivo sobre a renda.

Quantidade de Pessoas na Residência (0.2931): Ligeiro aumento em relação ao modelo anterior, sugerindo que residências com mais pessoas podem ter mais fontes de renda.

# Conclusões
O R2 ajustado permaneceu praticamente o mesmo, o que sugere que as variáveis removidas não contribuíam muito para a explicação da variabilidade na renda.

Tanto o AIC quanto o BIC diminuíram, o que geralmente é um bom sinal. Um valor menor desses critérios indica um melhor ajuste do modelo aos dados, considerando a complexidade do modelo.

Todos os p-valores das variáveis no modelo final são menores que 0.05, indicando que todas são estatisticamente significativas.